In [1]:
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
screen_width = 1920
screen_length = 1200 

cellx = 30 
celly = 40 

x = np.random.randint(1,1900,(100,1))
y = np.random.randint(1,1200,(100,1))
duration = np.random.randint(50,3000,(100,1))


dt =  np.zeros((100,3))
dt[:,:1] = x
dt[:,1:2] =  y 
dt[:,2:3] = duration

# AOI and Transition Matrix

In [3]:
np.random.randint(0,100,(5,3))

array([[14,  0, 17],
       [84,  9, 53],
       [ 7, 91,  0],
       [71, 37, 63],
       [33, 45, 99]])

In [4]:
from transition_matrix import *

#Setting important variables
TEST_SCREENDIM = [1920, 1200]
TEST_RADIUS = 100
TEST_VERTICES_1 = [[ 20,20] , [400,250], [40,400]]
TEST_VERTICES_2 = [[800,800], [1000,800], [700,650]]


#An AOI dict having all the important areas of interest that will be used to calculate different metrics
TEST_AOI_DICT = {
                'aoi_poly1': PolyAOI(TEST_SCREENDIM,TEST_VERTICES_1),
                'aoi_circle2': CircleAOI(TEST_SCREENDIM, TEST_RADIUS, center=[500,500]),
                'aoi_poly2': PolyAOI(TEST_SCREENDIM,TEST_VERTICES_2)
                }

#Dummy Gaze values
GAZE_ARRAY = np.array([
    [41, 41, 0],
    [80, 80, 0],
    [501, 501, 0],
    [100, 200, 0],
    [1002, 1002, 0],
    [400, 400, 0],
    [200, 200, 0],
    [250, 250, 0],
    [540, 540, 0]])

gz = GazeTransitions(TEST_SCREENDIM, TEST_AOI_DICT, GAZE_ARRAY)
print('The Gaze Transition Matrix\n',gz.get_transition_matrix(),'\n')
print('Number of Points outside all AOIs :',gz.points_outside_aoi)
for n,a in TEST_AOI_DICT.items():
    print('Probability of having Points in '+ n +': ',gz.get_prob_aoi(n))
print('Plot representation')
gz.plot_all(annotate_points=True)

The Gaze Transition Matrix
              aoi_poly1  aoi_circle2  aoi_poly2
aoi_poly1          2.0          2.0        0.0
aoi_circle2        1.0          0.0        0.0
aoi_poly2          0.0          0.0        0.0 

Number of Points outside all AOIs : 2
Probability of having Points in aoi_poly1:  0.5555555555555556
Probability of having Points in aoi_circle2:  0.2222222222222222
Probability of having Points in aoi_poly2:  0.0
Plot representation


<Figure size 1200x900 with 1 Axes>

# Entropy Metrics

For entropy two metrics will be calculated as mentioned in the Paper Krejtz et al., 2014

Gaze Transition Entropy:  
\begin{equation*}
    H_t =- \sum_i \pi_i \sum_j p_{ij}log p_{ij}
\end{equation*}

Gaze Stationary Entropy:

\begin{equation*}
    H_s =- \sum_i \pi_i log \pi_i
\end{equation*}

where  $p_{ij} = \frac{n_{ij}}{n}$  and $\pi_i = \frac{n_i}{n} $


 

In [8]:
from eyesmetriccalculator import EyesMetricCalculator

ec = EyesMetricCalculator(dt,GAZE_ARRAY,(screen_width,screen_length))

print('ConvexHullArea  : ' ,ec.convexHull('area').compute() )
print('SpatialDensity  : ' , ec.spatialDensity(cellx,celly).compute())
print('NNI Metric  : ' , ec.NNI().compute())
print('NNI Metric  : ' , ec.GEntropy(TEST_AOI_DICT).compute())

SyntaxError: invalid syntax (entropy.py, line 3)

In [ ]:
tp = np.array([   [0.78,0.06,0.16] ,
            [0.18,0.46,0.36],
             [0.15,0.09,0.76]
            ])

sp = np.array([0.42,0.14,0.44])

In [ ]:
tp

In [ ]:
total_ht = 0

for i in range(len(sp)):
    ent_sum = 0
    for j in range(len(tp)):
        ent_sum += tp[i,j] * np.log(tp[i,j])
    total_ht += sp[i] * ent_sum
total_ht = np.around(total_ht,decimals=2) * -1

In [ ]:
total_ht

In [ ]:
#calculating HS
s= 0
for j in sp:
        s +=j * np.log(j)
s= s * -1
s = np.around(s,decimals=2)
s

In [ ]:
gz.get_transition_matrix()/len(GAZE_ARRAY)

In [ ]:
statprob_list = []
for n,a in TEST_AOI_DICT.items():
    statprob_list.append(gz.get_prob_aoi(n))
statprob_list = np.array(statprob_list)

In [ ]:
statprob_list